# Geolocate a set from EO Master File


In [1]:
#!pip install -U googlemaps
import googlemaps
import pandas as pd

In [2]:
df = pd.DataFrame.from_csv('eo_ca.csv', index_col=None)

In [3]:
df['ADDRESS'] = df['STREET'] + ', ' + df['CITY'] + ', ' + df['STATE'] + ' ' + df['ZIP']
df.head()

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,...,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ADDRESS
0,2296179,RELIGIOUS SCIENCE CHURCH CENTER OF SAN DIEGO,NaN,4102 MARLBOROUGH,SAN DIEGO,CA,92105-1462,0,3,3,...,0,6,0,3,NaN,NaN,NaN,NaN,NaN,"4102 MARLBOROUGH, SAN DIEGO, CA 92105-1462"
1,3140260,PILGRIM BAPTIST CHURCH,NaN,217 N GRANT ST,SAN MATEO,CA,94401-1833,3125,3,9,...,0,6,0,12,NaN,NaN,NaN,NaN,1910,"217 N GRANT ST, SAN MATEO, CA 94401-1833"
2,3754390,LITTLE LEAGUE BASEBALL INC,% TONY GARCIA,36511 MILE END RD,SOLEDAD,CA,93960-9689,3158,3,9,...,0,1,0,9,NaN,NaN,NaN,NaN,4050914 SOLEDAD LL,"36511 MILE END RD, SOLEDAD, CA 93960-9689"
3,10104976,SAINT ANDREW CHURCH,NaN,4050 KATELLA AVE STE 101,LOS ALAMITOS,CA,90720-3463,0,3,3,...,0,6,0,12,NaN,NaN,NaN,X21Z,NaN,"4050 KATELLA AVE STE 101, LOS ALAMITOS, CA 907..."
4,10137902,AMERICAN FEDERATION OF STATE COUNTY & MUNICIPA...,% STEVEN FELDMAN,1535 RUGBY CIR,THOUSAND OAKS,CA,91360-3534,1381,5,9,...,0,2,0,12,0.0,0.0,0.0,NaN,L2006CA LA PROFESSIONAL MEDICAL EMP,"1535 RUGBY CIR, THOUSAND OAKS, CA 91360-3534"


In [4]:
df.dropna(axis=0, how='any', subset=['NTEE_CD'], inplace=True)

In [5]:
socialscienceOrgs = df.loc[df['NTEE_CD'].str.contains('V')]

In [6]:
socialscienceOrgs.shape

(200, 29)

In [7]:
gmaps = googlemaps.Client(key='ADD-YOUR-KEY-HERE')
geocode = gmaps.geocode(socialscienceOrgs.iloc[6]['ADDRESS'])
geocode[0]['geometry']['location']

{'lat': 32.7511561, 'lng': -117.1439528}

In [18]:
socialscienceOrgs['lat'] = ''
socialscienceOrgs['lng'] = ''
socialscienceOrgs.head()
socialscienceOrgs = socialscienceOrgs.reset_index() #Make sure to reset index or accomodate geolocation loop

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
# change Orgs of interest here so you won't have to change it 4x times in the loop
orgs = socialscienceOrgs
orgs.shape

(200, 32)

In [25]:
for i in range(0,len(orgs)):
    try:
        geocode = gmaps.geocode(orgs.iloc[i]['ADDRESS'])
        latlong = geocode[0]['geometry']['location']
        if geocode:
            orgs.loc[i, 'lat'] = latlong['lat']
            orgs.loc[i, 'lng'] = latlong['lng']
        else:
            orgs.loc[i, 'lat'] = ''
            orgs.loc[i, 'lng'] = ''
    except IndexError:
        geocode is None     

In [26]:
# Change name
orgs.to_csv('SocialScienceOrgs-locations.csv')

In [27]:
# add NTEE Taxonomy
df = pd.DataFrame.from_csv('NTEE-CODES-V2.csv')
dfmerged = pd.merge(orgs, df, how = 'inner', on = 'NTEE_CD')
dfmerged.shape # will lose special cases like 'V06E' intead of 'V06'

In [37]:
# append to growing list
dftodas = pd.DataFrame.from_csv('OOIV3.csv')
dftodas = dftodas.append(dfmerged, ignore_index=True)
dftodas.head()
dftodas.to_csv('OOIV3.csv')